In [5]:
! pip install -U langchain_community langchain-openai langchain-anthropic langchain langgraph bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.7/939.7 kB 4.2 MB/s eta 0:00:00 MB/s eta 0:00:01
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/home/fduser/Desktop/Langgraph/venv_mid_sem_exam/lib/python3.12/site-packages/README.rst'
Check the permissions.



In [1]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

OPENAI_API_KEY:  ········


In [6]:
from langchain import hub
from langchain.chat_models import ChatOpenAI
from langchain.tools import FileTool
from langchain.graph import StateGraph
from pydantic import BaseModel, Field
from typing import List

# Define the State model using Pydantic
class State(BaseModel):
    java_code: str = Field(default="")
    rubric: str = Field(default="")
    class_names: List[str] = Field(default_factory=list)
    evaluation_response: str = Field(default="")
    marks_list: str = Field(default="")
    total_marks: int = Field(default=0)
    final_evaluation: str = Field(default="")

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4-turbo-preview")

# Define the nodes for the StateGraph workflow
def read_java_file(state: State, java_file_path: str):
    """Reads Java code from the specified file and updates the state."""
    with open(java_file_path, 'r') as java_file:
        state.java_code = java_file.read()

def read_rubric_file(state: State, rubric_file_path: str):
    """Reads the rubric from the specified file and updates the state."""
    with open(rubric_file_path, 'r') as rubric_file:
        state.rubric = rubric_file.read()

def extract_class_names(state: State):
    """Extracts all class names used in the Java code and updates the state."""
    class_extraction_prompt = f"""
    I have the following Java code:
    ```
    {state.java_code}
    ```
    Please extract all class names used in the code.
    """
    class_response = llm(class_extraction_prompt)
    state.class_names = class_response.strip().split(',')

def evaluate_classes(state: State):
    """Evaluates each Java class based on the rubric and updates the state."""
    evaluation_prompt = f"""
    I have the following rubric/marking scheme:
    ```
    {state.rubric}
    ```
    I also have the following list of Java classes: {', '.join(state.class_names)}.
    Please evaluate each class based on the most relevant part of the rubric.
    Provide specific feedback for each class.
    """
    state.evaluation_response = llm(evaluation_prompt)

def re_evaluate(state: State):
    """Re-evaluates the previously obtained evaluation and updates the state."""
    re_evaluation_prompt = f"""
    Based on the previous evaluation, please provide a re-evaluation of the classes.
    Here is the initial evaluation:
    ```
    {state.evaluation_response}
    ```
    Provide detailed comments and additional insights if necessary.
    """
    state.evaluation_response = llm(re_evaluation_prompt)

def extract_marks(state: State, student_code_path: str):
    """Extracts marks from the evaluation response based on the student's Java code."""
    with open(student_code_path, 'r') as student_file:
        student_code = student_file.read()

    marks_extraction_prompt = f"""
    I have the following student's Java code:
    ```
    {student_code}
    ```

    And the following evaluation points for the student's performance:
    ```
    {state.evaluation_response}
    ```

    Please extract the marks awarded for each criterion in the evaluation as a comma-separated list of marks.
    """
    marks_response = llm(marks_extraction_prompt)
    state.marks_list = marks_response.strip()

def sum_marks(state: State):
    """Sums the extracted marks and updates the total marks in the state."""
    marks = [int(mark.strip()) for mark in state.marks_list.split(',') if mark.strip().isdigit()]
    state.total_marks = sum(marks)

def generate_final_evaluation(state: State):
    """Generates the final evaluation summary and updates the state."""
    final_evaluation_prompt = f"""
    I have extracted the following marks for the student: {state.marks_list}.

    Based on these marks, the total score is: {state.total_marks}.

    Now, please provide detailed feedback and a final assessment for the student's performance in their Java classes.
    """
    state.final_evaluation = llm(final_evaluation_prompt)

def save_final_evaluation(state: State):
    """Saves the final evaluation and total marks to a file."""
    with open('final_evaluation.txt', 'w') as eval_file:
        eval_file.write("Final Evaluation:\n")
        eval_file.write(state.final_evaluation)
        eval_file.write(f"\nTotal Marks Obtained: {state.total_marks}\n")

# Create a StateGraph to manage the workflow
graph = StateGraph()

# Instantiate the state model
state = State()

# Define the nodes in the graph workflow
graph.add_node('Read Java File', read_java_file, state, "/path/to/student_code.java")
graph.add_node('Read Rubric File', read_rubric_file, state, "/path/to/rubric_file.txt")
graph.add_node('Extract Class Names', extract_class_names, state)
graph.add_node('Evaluate Classes', evaluate_classes, state)
graph.add_node('Re-Evaluate Classes', re_evaluate, state)
graph.add_node('Extract Marks', extract_marks, state, "/path/to/student_code.java")
graph.add_node('Sum Marks', sum_marks, state)
graph.add_node('Generate Final Evaluation', generate_final_evaluation, state)
graph.add_node('Save Final Evaluation', save_final_evaluation, state)

# Connect the nodes to create the workflow
graph.connect('Read Java File', 'Read Rubric File')
graph.connect('Read Rubric File', 'Extract Class Names')
graph.connect('Extract Class Names', 'Evaluate Classes')
graph.connect('Evaluate Classes', 'Re-Evaluate Classes')
graph.connect('Re-Evaluate Classes', 'Extract Marks')
graph.connect('Extract Marks', 'Sum Marks')
graph.connect('Sum Marks', 'Generate Final Evaluation')
graph.connect('Generate Final Evaluation', 'Save Final Evaluation')

# Run the workflow
graph.run()
print("Final evaluation saved in 'final_evaluation.txt'.")


ImportError: cannot import name 'FileTool' from 'langchain.tools' (/home/fduser/Desktop/Langgraph/venv_mid_sem_exam/lib/python3.12/site-packages/langchain/tools/__init__.py)